# Schema.org Ontology Resource Templates

In [2]:
import datetime
import json
from collections import OrderedDict
import rdflib
import requests
from bs4 import BeautifulSoup
print(datetime.datetime.utcnow())

2019-06-12 18:22:26.327657


In [118]:
def init_profile(ident, title, description):
    profile = OrderedDict()
    profile["Profile"] = OrderedDict()
    profile["Profile"]["resourceTemplates"] = []
    profile["Profile"]["id"] = ident
    profile["Profile"]["date"] = datetime.datetime.utcnow().strftime("%Y-%m-%d")
    profile["Profile"]["description"] = description
    profile["Profile"]["author"] = "Jeremy Nelson"
    profile["Profile"]["title"] = title
    profile["Profile"]["schema"] = "https://ld4p.github.io/sinopia/schemas/0.1.0/profile.json"
    return profile

def init_resource_template(**kwargs):
    resource_template = OrderedDict()
    resource_template["id"] = kwargs.get("ident")
    resource_template["propertyTemplates"] = kwargs.get("propertyTemplates", [])
    resource_template["resourceURI"] = kwargs.get("uri")
    resource_template["resourceLabel"] = kwargs.get("label")
    resource_template["author"] = "Jeremy Nelson <https://orcid.org/0000-0002-7543-3893>"
    resource_template["date"] = datetime.datetime.utcnow().strftime("%Y-%m-%d")
    resource_template["schema"] = "https://ld4p.github.io/sinopia/schemas/0.1.0/resource-template.json"
    return resource_template

In [107]:
def get_soup(url):
    return BeautifulSoup(requests.get(url).text)
    
def property_template(tr):
    property_template = { "propertyURI": tr.attrs["resource"],
                          "mandatory": "false",
                          "repeatable": "true",
                          "type": "literal"
                        }
    table_head = tr.find_all("th")
    table_data = tr.find_all("td")
    raw_label = table_head[0].text.strip()
    label = []
    for i, char in enumerate(raw_label):
        if i > 0 and char.isupper():
            label.append(f" {char}")
        else:
            label.append(char)
    property_template["propertyLabel"] = ''.join(label).title()
    if len(table_data) > 1:
        property_template["remark"] = table_data[1].text
    return property_template
   
    
    

In [2]:
with open('/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaThingProfile.json') as fo:
    thing = json.load(fo)

In [3]:
for key in thing.keys():
    print(key)

Profile


## Create Work Profile 2019-06-10 &amp; 2019-06-13

In [119]:
creative_work_profile = init_profile("profile:schema:creative_work", 
                                     "Schema.org Creative Work Profile", 
                                    """The most generic kind of creative work, including books, movies, 
                                    photographs, software programs, etc.""")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
creative_work_resource_template = init_resource_template(ident="sinopia:resourceTemplate:schema:CreativeWork",
                                                         propertyTemplates=property_templates,
                                                         uri="http://schema.org/CreativeWork",
                                                         label="Creative Work")

creative_work_soup = get_soup("https://schema.org/CreativeWork")
def_table = creative_work_soup.find("table")
def_table_rows = def_table.find_all('tr')

In [120]:
for row in def_table_rows[2:]:
    try:
        creative_work_resource_template["propertyTemplates"].append(
            property_template(row))
    except:
        break

In [110]:
print(f"Length: {len(creative_work_resource_template['propertyTemplates'])}")

Length: 90


In [121]:
creative_work_profile["Profile"]["resourceTemplates"].append(creative_work_resource_template)

In [122]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaCreativeWork.json", "w+") as fo:
    json.dump(creative_work_profile, fo, indent=2, sort_keys=True)

## Book Profile 2019-06-10

In [125]:
book_profile = init_profile("sinopia:profile:schema:Book", "Schema.org Book Profile", "A Book")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  },
  {
    "mandatory": "false",
    "propertyLabel": "CreativeWork Properties",
    "propertyURI": "http://schema.org/CreativeWork",
    "type": "resource",
    "valueConstraint": {
    "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:CreativeWork"
        ]
    }
  }
]
book_resource_template = init_resource_template(ident="sinopia:resourceTemplate:schema:Book", 
                                                propertyTemplates=property_templates, uri="http://schema.org/Book", label="Book")
book_profile['Profile']['resourceTemplates'].append(book_resource_template)

In [123]:
book_soup = get_soup("https://schema.org/Book")
first_table = book_soup.find("table")

In [126]:
table_rows = first_table.find_all("tr")
for row in table_rows[2:8]:
    book_resource_template["propertyTemplates"].append(
        property_template(row)
    )

In [127]:
print(table_rows[2].find_all("td")[1].text)

Indicates whether the book is an abridged edition.


In [128]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaBookProfile.json", "w+") as fo:
    json.dump(book_profile, fo, indent=2, sort_keys=True)

## Person Profile 2019-06-12 &amp; 2019-06-13

In [69]:
person_profile = init_profile("sinopia:profile:schema:Person", 
                              "Schema.org Person Profile", 
                              "A person (alive, dead, undead, or fictional).")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
person_rt = init_resource_template(
    ident="sinopia:resourceTemplate:schema:Person",
    propertyTemplates=property_templates, 
    uri="http://schema.org/Person",
    label="A person (alive, dead, undead, or fictional).")

In [129]:
person_soup = get_soup("https://schema.org/Person")
first_table = person_soup.find("table")
table_rows = first_table.find_all("tr")

In [73]:
len(person_rt["propertyTemplates"])

54

In [130]:
for row in table_rows[2:55]:
    try:
        person_rt["propertyTemplates"].append(property_template(row))
    except:
        break
    

In [74]:
person_profile["Profile"]['resourceTemplates']

[]

In [131]:
person_profile["Profile"]['resourceTemplates'].append(person_rt)

In [132]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaPersonProfile.json", "w+") as fo:
    json.dump(person_profile, fo, indent=2, sort_keys=True)

## Event Profile 06-13-2019

In [143]:
event_profile = init_profile("sinopia:profile:schema:Event",
                            "Schema.org Event Profile",
                            "An event realism, such as a concert, lecture, or festival.")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
event_rt = init_resource_template(ident="sinopia:resourceTemplate:schema:Event",
                                  propertyTemplates=property_templates,
                                  uri="http://schema.org/Event",
                                  label="Event")

In [144]:
event_soup = get_soup("https://schema.org/Event")
first_table = event_soup.find("table")
table_rows = first_table.find_all("tr")

In [145]:
print(table_rows[2], table_rows[33])
print(f"Length {len(event_rt['propertyTemplates'])}")

<tr resource="http://schema.org/about" typeof="rdfs:Property">
<th class="prop-nam" scope="row">
<code property="rdfs:label"><a href="./about">about</a></code>
</th>
<td class="prop-ect">
<link href="http://schema.org/Thing" property="rangeIncludes"/><a href="./Thing">Thing</a> <link href="http://schema.org/CommunicateAction" property="domainIncludes"/><link href="http://schema.org/CreativeWork" property="domainIncludes"/><link href="http://schema.org/Event" property="domainIncludes"/></td><td class="prop-desc" property="rdfs:comment">The subject matter of the content.<br/> Inverse property: <a href="./subjectOf">subjectOf</a>.</td></tr> <tr resource="http://schema.org/workPerformed" typeof="rdfs:Property">
<th class="prop-nam" scope="row">
<code property="rdfs:label"><a href="./workPerformed">workPerformed</a></code>
</th>
<td class="prop-ect">
<link href="http://schema.org/CreativeWork" property="rangeIncludes"/><a href="./CreativeWork">CreativeWork</a> <link href="http://schema.org/

In [146]:
for row in table_rows[2:33]:
    event_rt["propertyTemplates"].append(property_template(row))

In [147]:
print(f"Length {len(event_rt['propertyTemplates'])}")
event_profile["Profile"]["resourceTemplates"].append(event_rt)

Length 32


In [149]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaEvent.json", "w+") as fo:
    json.dump(event_profile, fo, indent=2, sort_keys=True)

## Organization Profile 06-14-2019

In [151]:
org_profile = init_profile("sinopia:profile:schema:Organization",
                           "Schema.org Organization",
                          "An organization such as a school, NGO, corporation, club, etc.")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
org_rt = init_resource_template(ident="sinopia:resourceTemplate:schema:Organization",
                                  propertyTemplates=property_templates,
                                  uri="http://schema.org/Organization",
                                  label="Organization")

In [156]:
org_soup = get_soup("https://schema.org/Organization")
first_table = org_soup.find("table")
table_rows = first_table.find_all("tr")

In [161]:
for row in table_rows[2:52]:
    org_rt["propertyTemplates"].append(property_template(row))

In [162]:
org_profile["Profile"]["resourceTemplates"].append(org_rt)

In [163]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaOrganization.json", "w+") as fo:
    json.dump(org_profile, fo, indent=2, sort_keys=True)

## Action Profile 06-14-2019

In [164]:
action_profile = init_profile("sinopia:profile:schema:Action",
                              "Schema.org Action",
                              "An action performed by a direct agent and indirect participants upon a direct object.")
property_templates = [
   {
    "mandatory": "false",
    "propertyLabel": "Thing Properties",
    "propertyURI": "http://schema.org/Thing",
    "repeatable": "true",
    "type": "resource",
    "valueConstraint": {
      "valueTemplateRefs": [
        "sinopia:resourceTemplate:schema:Thing"
      ]
    }
  }
]
action_rt = init_resource_template(ident="sinopia:resourceTemplate:schema:Action",
                                   propertyTemplates=property_templates,
                                   uri="http://schema.org/Action",
                                   label="Action")

In [165]:
action_soup = get_soup("https://schema.org/Action")
first_table = action_soup.find("table")
table_rows = first_table.find_all("tr")

In [168]:
print(table_rows[2], table_rows[12])

<tr resource="http://schema.org/actionStatus" typeof="rdfs:Property">
<th class="prop-nam" scope="row">
<code property="rdfs:label"><a href="./actionStatus">actionStatus</a></code>
</th>
<td class="prop-ect">
<link href="http://schema.org/ActionStatusType" property="rangeIncludes"/><a href="./ActionStatusType">ActionStatusType</a> <link href="http://schema.org/Action" property="domainIncludes"/></td><td class="prop-desc" property="rdfs:comment">Indicates the current disposition of the Action.</td></tr> <tr resource="http://schema.org/target" typeof="rdfs:Property">
<th class="prop-nam" scope="row">
<code property="rdfs:label"><a href="./target">target</a></code>
</th>
<td class="prop-ect">
<link href="http://schema.org/EntryPoint" property="rangeIncludes"/><a href="./EntryPoint">EntryPoint</a> <link href="http://schema.org/Action" property="domainIncludes"/></td><td class="prop-desc" property="rdfs:comment">Indicates a target EntryPoint for an Action.</td></tr>


In [169]:
for row in table_rows[2:12]:
    action_rt["propertyTemplates"].append(property_template(row))


In [171]:
action_profile["Profile"]["resourceTemplates"].append(action_rt)

In [172]:
with open("/Users/jpnelson/2019/sinopia_sample_profiles/profiles/v0.1.0/SchemaAction.json", "w+") as fo:
    json.dump(action_profile, fo, indent=2, sort_keys=True)